# Mapping Murdock's Ethnographic Atlas
## using geopandas and folium 
### (relevant to a replication of James Fenske's paper)

This notebook illustrates how to do mapping and some geospatial analysis inside of a juptyer notebook using various python libraries.  All of these steps can of course be done inside of dedicated GIS client such as QGIS but it's nice to be able to do many of the same tasks inside of a notebook for better documentation and automation. 

The notebook illustrates how to load and display vector polygon as well as raster geospatial data. There are many ways to do this with python but we focus here on using [geopandas](https://github.com/geopandas/geopandas) with [folium](http://python-visualization.github.io/folium/) in a supporting role (to make things look more cool on interactive slippy maps using [leaflet.js](http://leafletjs.com/)).  We use the [rasterio](https://github.com/mapbox/rasterio) and [rasterstats](http://pythonhosted.org/rasterstats/) libraries to work with raster data. 

Broad outline:

1. load vector polygon from [Morduck's Ethnographic Atlas](http://devecondata.blogspot.com/2007/10/ethnographic-atlas.html)  data digitized by Nathan Nunn. We'll display the data and show how to also plot markers on a different map layer.

2. Load and display gridded raster data on wheat suitability. 

3. Calculate raster zonal statistics 

4. (not finished) Discuss how data such as that constructed in step 3 was used in an empirical paper by James Fenske...

In [1]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import seaborn as sns
from zipfile import ZipFile
plt.style.use('bmh')

First create a folium slippy basemap. 

In [2]:
m = folium.Map(
    location = [-1, 25], 
    zoom_start=4, 
    tiles='cartodbpositron'
)

We use `FeatureGroup`s to add feature layers that we can later turn off and on via LayerControl

We will have different folium map layers or FeatureGroups. 

The `marker_group` below is just a collection of markers (all appear at once). 
The `marker_cluster` will, depending on zoom level, cluster markers into groups to make the map less cluttered. We turn on only one of the marker feature groups at a time.

The polygon group will have tribal group borders. 

In [3]:
poly_group = folium.FeatureGroup(name='Tribes')
marker_group = folium.FeatureGroup(name='Markers')
marker_cluster = MarkerCluster().add_to(m)

#### Morduck's Ethnographic Atlas
Shapefile [digitized by Nathan Nunn](https://scholar.harvard.edu/nunn/pages/data-0) 
We import this shapefile into a geopandas dataframe:

In [4]:
!dir *.zip

 Volume in drive C is Windows
 Volume Serial Number is A040-41A7

 Directory of C:\Users\jconn\Google Drive\GC\land\notebooks



File Not Found


In [9]:
shpfile = r'zip://borders_tribes.zip'
tribe_areas = gpd.GeoDataFrame.from_file(shpfile)
tribe_areas.head(3)

OSError: no such file or directory: 'borders_tribes.zip'

geopandas is a powerful python library. Among many other things it allows us to plot:

In [ ]:
tribe_areas.plot(column='LAT', figsize=(12,16));

But this is a static map. For exploratory purposes it's nice to pass the information in the dataframe to folium to map things on top of a slippy map.  

The following adds the polygon information from our geopandas dataframe to the `poly_group` map layer that we will use below with folium.

In [ ]:
folium.GeoJson(tribe_areas).add_to(poly_group);

##### Markers
We'll iterate through the ethnic groups and create a marker for each group to be located at the Lat, Lon position recorded in the geopandas data frame and labeled with the ethnic group name. 

In [ ]:
for i, row in tribe_areas.iterrows():
    folium.Marker(
        location=[row.LAT, row.LON],
        popup = row.NAME).add_to(marker_cluster)  # or replace with marker_group

We choose which of these FeatureGroups to add to the folium map and add a `LayerControl` to allow us to selectively turn these layers on or off.

**NOTE**: the map that follows will not be seen in static views of this notebook (as you might see on github). See this [saved HTML rendering](https://cdn.rawgit.com/jhconning/land_uct_2018/cbc64c77/notebooks/tribes.html) (which should be interactive)

In [ ]:
marker_group.add_to(m)
#marker_cluster.add_to(m)
poly_group.add_to(m)

folium.LayerControl().add_to(m)

m

Save the map to an HTML file to be browsed as a standalone in a larger window.

In [ ]:
m.save('tribes.html')

## Raster data, zonal statistics

We want to calculate zonal statistics (in this case statistics for data that falls inside the ethnic group boundaries).  Here we'll get data on wheat yield potential which comes as raster data from FAO-GAEZ. Each gridpoint on the raster is associated with a suitability index.  We'll calculate mean suitability in each of the polygons associated with Murdock/Nunn data.

In [ ]:
import rasterio
import rasterio.plot as rioplot
import rasterstats as rs

Get first downloaded raster data on potential crop yields from FAO-GAEZ data.  Looks like this. 

In [ ]:
wheat = rasterio.open("res02_crav6190h_whea000a_yld.tif")
plt.figure(figsize=(12,10))
plt.xlim(1800,2900)
plt.ylim(1600,600)
rioplot.show(wheat);

Next we use the `rasterstats.zonal_stats` command to calculate 'count', 'max', 'min', 'mean' statistics for the raster data inside each polygon. The function returns the results as a python dictionary.

In [ ]:
stats = rs.zonal_stats('borders_tribes.shp','res02_crav6190h_whea000a_yld.tif', prefix = 'wheat_')

Finally let us add these calculated statistics as columns to our geodataframe.

In [ ]:
colnames = pd.DataFrame(stats).columns.tolist()
colnames

In [ ]:
tribe_areas[colnames] = pd.DataFrame(stats)
tribe_areas.head()

Let's see a simple chloropeth plot based on mean wheat potential in each ethnic group zone.

In [ ]:
tribe_areas.plot(column='wheat_mean', figsize=(12,16));

## James Fenske's regressions

    Fenske, James. 2013. “Does Land Abundance Explain African Institutions?” The Economic Journal 123 (573):1363–1390.

To his great credit James Fenske posts data and code to replicate most of his published papers to his [website](https://sites.google.com/site/jamesfenske/Research)

Several of his right hand variables in the regressions in tables X, Y Z were constructed using methods such as those described above.


More to follow...